# Generate GIF area detector images
### (Under construction)
Set range of frames to use
Set total time for GIF

Set ROI
Plot first and last with ROI - and color scale

Calculate sequence of frames based on range and total time at 20 FPS



In [ ]:
%matplotlib widget
import DanMAX as DM
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import PIL 
import os
from ipywidgets import interact, widgets
style = DM.darkMode(style_dic={'size' : 'large', 'grid' : False})

In [ ]:
fname = DM.findScan()

# Range of frames to be included in the gif
first_frame = 100
last_frame = 1000

gif_length = 2 # Total length in seconds

dset = '/entry/instrument/pilatus/data'
with h5.File(fname,'r') as f:
    frame0 = f[dset][first_frame]
    frame1 = f[dset][last_frame]

fig, ax = plt.subplots()
crop = ax.imshow(frame0)

def update_crop(top=0, down=frame0.shape[0], left=0, right=frame0.shape[1]):
    for child in ax.get_children():
        if type(child) == PolyCollection:
             child.remove()
    ax.fill_between([0, frame0.shape[1]], 0, top, linewidth = 0, color='black', alpha=.5)
    ax.fill_between([0, frame0.shape[1]], frame0.shape[0], down, linewidth = 0, color='black', alpha=.5)
    ax.fill_betweenx([top, down], 0, left, linewidth = 0, color='black', alpha=.5)
    ax.fill_betweenx([top, down], frame0.shape[1], right, linewidth = 0, color='black', alpha=.5)
    return top, down, left, right

crop = interact(update_crop, top=(0, frame0.shape[0]), down=(0, frame0.shape[0]), left=(0, frame0.shape[1]), right=(0, frame0.shape[1]))

In [ ]:
# Results from ROI selection
top, down, left, right = crop.widget.result
roi = [(top, down), (left,right)] 

f, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(frame0[roi[0][0]:roi[0][1],roi[1][0]:roi[1][1]])
ax1.set_title('First frame')
ax2.imshow(frame1[roi[0][0]:roi[0][1],roi[1][0]:roi[1][1]])
ax2.set_title('Last frame')

In [ ]:
n_frames = last_frame-first_frame
n_required_frames = gif_length*20
ratio = round(n_frames/n_required_frames)

# import images from the .h5 file
with h5.File(fname,'r') as f:
    images = f[dset][first_frame:last_frame:ratio]
print('{} frames has been loaded'.format(len(images)))


In [ ]:
gif_fname = f'gif_{os.path.splitext(os.path.basename(fname))[0]}.gif'

estVmax = np.mean(images[:,roi[0][0]:roi[0][1],roi[1][0]:roi[1][1]])*2

imgs = [PIL.Image.fromarray(np.uint8(plt.cm.viridis(np.clip(img[roi[0][0]:roi[0][1],roi[1][0]:roi[1][1]],0, estVmax)/estVmax)*255)) for img in images]

imgs[0].save(gif_fname, save_all=True, append_images=imgs[1:], duration=gif_length, loop=0)